# Librerias

In [111]:
# Librerias

import requests
import os
from dotenv import load_dotenv
import sqlite3
import ast

import pandas as pd

#Machine Learning

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from pickle import dump
load_dotenv()

True

# API REQUESTS

In [93]:
#API vars

API_KEY = os.getenv("API_KEY")
MAIN_URL = os.getenv("MAIN_URL")

#Cogemos las 5000 mejores peliculas actuales de TMDB

def GetBest5KMovies(key, url, movies = 500):
    moviePages = 20 #20 peliculas por "pagina", lo pongo en una variable por si en algun momento se tiene que reutilizar el codigo para otra API.
    necesariesrequests = int(movies/moviePages)
    data_movies = []
    data_credits = []
    dict_genre = {}
    
    #Creamos el dataset de las peliculas.
    for p in range(1, necesariesrequests +1):
        full_url = f"{url}movie/popular?api_key={key}&page={p}"

        response = requests.get(full_url)
        
        if response.status_code == 200:
            print("Conexión Exitosa")
            data_movie = response.json()
            data_movies.extend(data_movie["results"]) #Extend es mejor que append porque ya ordena el dataset.
            print(f"pagina {p}")
        
        else:
            print(f"Error: {response.status_code}")
            return None
    
    #Creamos el dataset de los credits
    
    for p in range(1, necesariesrequests+1):
        full_url = (f"{url}movie/popular?api_key={key}&page={p}")
        
        response = requests.get(full_url)
        
        if response.status_code == 200:
            print("Conexion Exitosa")
            
            data_credit = response.json()
            
            for m in data_credit["results"]:
                movie_id = m["id"]
                title = m["title"]
                overview = m["overview"]
                genre_ids = m["genre_ids"]
                cast = [] #Inicializamos
                crew = [] #Inicializamos
                keyword = [] #Inicializamos
                
                #Obtenemos los datos del cast.
                cast_url = (f"{url}movie/{movie_id}/credits?api_key={key}")
                response = requests.get(cast_url)
                    
                if response.status_code == 200:
                    print("Obteniendo datos del Cast...")
                    data_cast = response.json()
                    cast = [actor["name"] for actor in data_cast.get("cast", [])]
                    crew = [member["name"] for member in data_cast.get("crew", [])]
                    
                #Obtenemos las keywords.
                keyword_url = (f"{url}movie/{movie_id}/keywords?api_key={key}")
                response = requests.get(keyword_url)
                
                if response.status_code == 200:
                    print("Obteniendo palabras clave...")
                    data_words = response.json()
                    keyword = [keyword["name"] for keyword in data_words.get("keywords", [])]
                
                #Añadimos todo junto para no descuadrarlo.
                data_credits.append({
                    "movie_id" : movie_id,
                    "title" : title,
                    "overview" : overview,
                    "genre_ids" : genre_ids,
                    "cast" : cast,
                    "crew" : crew,
                    "keyword" : keyword
                })
        
    genre_url = (f"{url}genre/movie/list?api_key={key}") #URL de los generos para poder parsearlos con su id.
        
    response = requests.get(genre_url)
        
    if response.status_code == 200:
        print("Obteniendo Diccionario de generos.")
        data_genre = response.json()
        lst_genre = list(data_genre.values())[0] #La API Nos devuelve todos los generos dentro de una lista. Vamos a obtener la lista limpia.
        for g in lst_genre:
            dict_genre[g["id"]] = g["name"] #Fuerzo a coger la id como key y el name como valor, no me salia con update. No me acaba de gustar después buscar más info.
        
    
    
    #Convertimos la lista en un DataSet
    
    ds_movies = pd.DataFrame(data_movies)
    
    ds_credits = pd.DataFrame(data_credits)
    
    #Parseo de generos.
    ds_credits["genres"] = ds_credits["genre_ids"].apply(lambda x: list(map(dict_genre.get, x)) if isinstance(x, list) else ([])) #isinstance nos verifica si el valor se puede iterar (Vamos, que si es una lista o no)
    
    
    return ds_movies, ds_credits

In [94]:
data_movies, data_credits = GetBest5KMovies(API_KEY, MAIN_URL)

Conexión Exitosa
pagina 1
Conexión Exitosa
pagina 2
Conexión Exitosa
pagina 3
Conexión Exitosa
pagina 4
Conexión Exitosa
pagina 5
Conexión Exitosa
pagina 6
Conexión Exitosa
pagina 7
Conexión Exitosa
pagina 8
Conexión Exitosa
pagina 9
Conexión Exitosa
pagina 10
Conexión Exitosa
pagina 11
Conexión Exitosa
pagina 12
Conexión Exitosa
pagina 13
Conexión Exitosa
pagina 14
Conexión Exitosa
pagina 15
Conexión Exitosa
pagina 16
Conexión Exitosa
pagina 17
Conexión Exitosa
pagina 18
Conexión Exitosa
pagina 19
Conexión Exitosa
pagina 20
Conexión Exitosa
pagina 21
Conexión Exitosa
pagina 22
Conexión Exitosa
pagina 23
Conexión Exitosa
pagina 24
Conexión Exitosa
pagina 25
Conexion Exitosa
Obteniendo datos del Cast...
Obteniendo palabras clave...
Obteniendo datos del Cast...
Obteniendo palabras clave...
Obteniendo datos del Cast...
Obteniendo palabras clave...
Obteniendo datos del Cast...
Obteniendo palabras clave...
Obteniendo datos del Cast...
Obteniendo palabras clave...
Obteniendo datos del Cast..

# Guardamos los CSV.

- Este paso no es necesario realmente pero para evitar estar llamando a la API de TMDB los guardamos.

In [95]:
def SaveDataToCSV(dataset,filename):
    dataset.to_csv(f"../data/raw/{filename}.csv", sep=";", index=False)

In [96]:
SaveDataToCSV(data_movies, "data_movies")
SaveDataToCSV(data_credits, "data_credits")

# Conectamos la DB

In [97]:
def CreateDB():
    table_movies = pd.read_csv("../data/raw/data_movies.csv", sep=";")
    table_credits = pd.read_csv("../data/raw/data_credits.csv", sep=";")
    
    #Conectamos
    con = sqlite3.connect("../data/movies.db")
    
    table_movies.to_sql("movies", con, if_exists = "replace", index=False)
    table_credits.to_sql("credits", con, if_exists = "replace", index=False)
    
    #Unimos los dos datasets
    
    query = "SELECT * FROM movies INNER JOIN credits ON movies.id = credits.movie_id;"
    
    final_data = pd.read_sql_query(query, con)
    con.close()
    
    return final_data
    
    

In [98]:
data_final = CreateDB()

# Vamos a limpiar el Dataset y quedarnos con las columnas que nos piden.

In [99]:
#Upsi, parece que la carga que hice desde la API eran las columnas que nos pedian. No pasa nada, limpiamos igual.
clear_list = ["movie_id", "title", "overview", "genres", "keyword", "cast", "crew"]

def ClearData(dataset, lst):
    
    dataset.drop(columns = [col for col in dataset.columns if col not in lst], inplace = True) #not in
    #Vamos a imprimir todas las columnas a ver que nos devuelve
    print(dataset.columns)
    
    dataset = dataset.T.drop_duplicates().T #Hacemos la transposicion para poder borrar las filas repetidas y las volvemos a poner como columnas.
    
    print(dataset.columns)
    
    
    return dataset

In [100]:
data_final_clean = ClearData(data_final, clear_list)

Index(['overview', 'title', 'movie_id', 'title', 'overview', 'cast', 'crew',
       'keyword', 'genres'],
      dtype='object')
Index(['overview', 'title', 'movie_id', 'cast', 'crew', 'keyword', 'genres'], dtype='object')


In [101]:
data_final_clean

,overview,title,movie_id,cast,crew,keyword,genres
0,"Sonic, Knuckles, and Tails reunite against a p...",Sonic the Hedgehog 3,939243,"['Jim Carrey', 'Ben Schwartz', 'Keanu Reeves',...","['Neal H. Moritz', 'Jeff Fowler', 'Brandon Tro...","['moon', 'sequel', 'based on video game', 'dua...","['Action', 'Science Fiction', 'Comedy', 'Family']"
1,Kraven Kravinoff's complex relationship with h...,Kraven the Hunter,539972,"['Aaron Taylor-Johnson', 'Ariana DeBose', 'Fre...","['Ben Davis', 'Art Marcum', 'Matt Holloway', '...","['hunter', 'superhero', 'villain', 'based on c...","['Action', 'Science Fiction', 'Adventure', 'Fa..."
2,Fifteen years after vanishing from the CIA to ...,Back in Action,993710,"['Cameron Diaz', 'Jamie Foxx', 'McKenna Robert...","['Janelle Beaudry', 'Seth Gordon', ""Brendan O'...","['spy', 'gun fight', 'assertive', 'spy action']","['Action', 'Comedy']"
3,A gothic tale of obsession between a haunted y...,Nosferatu,426063,"['Lily-Rose Depp', 'Nicholas Hoult', 'Bill Ska...","['Jan Cajzl', 'Pavel Cajzl', 'Miroslav Lhotka'...","['cold', 'based on novel or book', 'vampire', ...","['Horror', 'Fantasy']"
4,After receiving an unexpected call from her wa...,Moana 2,1241982,"['Auliʻi Cravalho', 'Dwayne Johnson', 'Hualāla...","['David G. Derrick Jr.', 'Abigail Barlow', 'Em...","['sea', 'ocean', 'villain', 'musical', 'sequel...","['Animation', 'Adventure', 'Family', 'Comedy',..."
...,...,...,...,...,...,...,...
663,Exceptionally ambitious high schooler Matthew ...,The Girl Next Door,10591,"['Emile Hirsch', 'Elisha Cuthbert', 'Timothy O...","['Luke Greenfield', 'Traci Kirshbaum', 'Arnon ...","['high school', 'porn actor', 'pornographic vi...","['Comedy', 'Romance']"
664,Yeong-tak and Ha-na are stepsiblings who live ...,Purpose of Tutoring,1231818,"['Ga Won', 'Han Seo-ah', 'Cha Myung-Hoon', 'Wo...",['Han Dong-yeon'],"['softcore', 'nudity']","['Romance', 'Drama']"
665,Yeong-tak and Ha-na are stepsiblings who live ...,Purpose of Tutoring,1231818,"['Ga Won', 'Han Seo-ah', 'Cha Myung-Hoon', 'Wo...",['Han Dong-yeon'],"['softcore', 'nudity']","['Romance', 'Drama']"
666,A well-to-do husband who allows his wife to ha...,Deep Water,619979,"['Ben Affleck', 'Ana de Armas', 'Tracy Letts',...","['Eigil Bryld', 'Marco Beltrami', 'Andrew Mond...","['infidelity', 'jealousy', 'based on novel or ...","['Drama', 'Mystery', 'Thriller']"


# Creamos la columna Tags

- Vamos a combinar las columnas de cast, crew, keywords, genres y overviews en solo una llamada tags.

In [102]:
def MergeTags (dataset):
    #Convertimos las columnas en listas, separamos las palabras
    dataset["overview"] = dataset["overview"].apply(lambda x: [x] if isinstance(x, str) else(x)) #Con el pd.isna() me da error de ambigüedad, con isinstance no.
    dataset["cast"] = dataset["cast"].apply(lambda x: ast.literal_eval(x)) #Convertimos la cadena de texto en una lista real.
    dataset["crew"] = dataset["crew"].apply(lambda x: ast.literal_eval(x))
    dataset["keyword"] = dataset["keyword"].apply(lambda x: ast.literal_eval(x))
    dataset["genres"] = dataset["genres"].apply(lambda x: ast.literal_eval(x))
    
    #Creamos la columna tag
    dataset["tag"] = dataset["overview"] + dataset["cast"] + dataset["crew"] + dataset["keyword"] + dataset["genres"]
    
    #Eliminamos las otras columnas.
    dataset.drop(columns = ["overview", "cast", "crew", "keyword", "genres"], inplace = True)
    
    #Convertimos tag en strings
    dataset["tag"] = dataset["tag"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)  #Separamos por espacio para pasarle el vectorizador  
    return dataset

In [103]:
data_with_tags = MergeTags(data_final_clean)

In [104]:
data_with_tags

,title,movie_id,tag
0,Sonic the Hedgehog 3,939243,"Sonic, Knuckles, and Tails reunite against a p..."
1,Kraven the Hunter,539972,Kraven Kravinoff's complex relationship with h...
2,Back in Action,993710,Fifteen years after vanishing from the CIA to ...
3,Nosferatu,426063,A gothic tale of obsession between a haunted y...
4,Moana 2,1241982,After receiving an unexpected call from her wa...
...,...,...,...
663,The Girl Next Door,10591,Exceptionally ambitious high schooler Matthew ...
664,Purpose of Tutoring,1231818,Yeong-tak and Ha-na are stepsiblings who live ...
665,Purpose of Tutoring,1231818,Yeong-tak and Ha-na are stepsiblings who live ...
666,Deep Water,619979,A well-to-do husband who allows his wife to ha...


# MACHINE LEARNING

In [105]:
data_with_tags.isna().any()

title       False
movie_id    False
tag          True
dtype: bool

In [106]:
data_with_tags.shape

(668, 3)

In [107]:
def TextVectorizor(dataset, col):
    #Primero nos aseguramos que el tag no tiene ningun NaN. Si tiene NaN la eliminaremos (No podemos recomendar sin los tags)
    dataset = dataset.dropna(subset=[col])
    print(dataset.shape)
    
    #Eliminamos los duplicados.
    dataset = dataset.drop_duplicates(subset = "title")
    print(dataset.shape)
    
    #Una vez limpio, vectorizamos
    vectorizor = TfidfVectorizer()
    col_vect = vectorizor.fit_transform(dataset[col])
    
    return col_vect

In [108]:
fit_vect = TextVectorizor(data_with_tags, "tag")

fit_vect.shape

(663, 3)
(414, 3)


(414, 39501)

In [109]:
model = NearestNeighbors(n_neighbors= 6, algorithm="brute", metric="cosine")
model.fit(fit_vect)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [110]:
movie_title = "Kraven the Hunter"
similarity = cosine_similarity(fit_vect)

selected_movie = data_with_tags[data_with_tags["title"] == movie_title].index[0]
dst = similarity[selected_movie]

movie_lst = sorted(list(enumerate(dst)), reverse = False , key = lambda x: x[1])[1:6]

for m in movie_lst:
        print(data_with_tags.iloc[m[0]].title)

Paddington in Peru
Yo no soy esa
The Garfield Movie
Moana
Squid Game: Fireplace


In [112]:
dump(model, open("../models/knn_neighbors.sav", "wb"))